### **1. Import Library**

In [2]:
import pandas as pd
import numpy as np
import os

print("Library diimport.")

Library diimport.


### **2. Path**

In [3]:
path_data_raw = "../data_raw/"
path_data_clean = "../data_clean/"

# Buat folder data_clean jika belum ada
os.makedirs(path_data_clean, exist_ok=True)

print(f"Path data raw: {path_data_raw}")
print(f"Path data clean: {path_data_clean}")

Path data raw: ../data_raw/
Path data clean: ../data_clean/


### **3. Penggabungan**

In [5]:
years = [2021, 2022, 2023]
all_dfs = []

print(f"Memulai proses untuk tahun: {years}...")

for year in years:
    print(f"\n--> Memproses Tahun {year}...")
    
    # 1. Variabel X1-X4 (UHH, HLS, RLS, Pengeluaran)
    # File ini menggunakan "Kabupaten / Kota"
    file_ipm = f"{path_data_raw}[Metode Baru] Indeks Pembangunan Manusia Menurut Kabupaten_Kota, {year}.csv"
    df_ipm = pd.read_csv(file_ipm) 
    df_ipm = df_ipm.iloc[3:].reset_index(drop=True) 
    df_ipm = df_ipm[[
        "Kabupaten / Kota", # Nama kolom ASLI
        "Unnamed: 1",       # UHH
        "Unnamed: 2",       # HLS
        "Unnamed: 3",       # RLS
        "Unnamed: 4"        # Pengeluaran
    ]]
    df_ipm = df_ipm.rename(columns={
        "Kabupaten / Kota": "Wilayah", 
        "Unnamed: 1": "UHH",
        "Unnamed: 2": "HLS",
        "Unnamed: 3": "RLS",
        "Unnamed: 4": "Pengeluaran"
    })

    
    # 2. Variabel Y (P1 - Indeks Kedalaman Kemiskinan)
    # File ini menggunakan "Kabupaten / Kota"
    file_p1 = f"{path_data_raw}Indeks Kedalaman Kemiskinan (P1) Menurut Kabupaten_Kota di Provinsi Jawa Tengah, {year}.csv"
    df_p1 = pd.read_csv(file_p1) 
    
    if "Indeks Kedalaman Kemiskinan" in df_p1.columns:
        # file P1(2021)
        df_p1 = df_p1.iloc[3:].reset_index(drop=True)
        df_p1 = df_p1.rename(columns={"Unnamed: 0": "Wilayah", "Indeks Kedalaman Kemiskinan": "P1"})
        df_p1 = df_p1[['Wilayah', 'P1']]
    else:
        # file P1(2023) atau P1(2022)
        df_p1 = df_p1.iloc[2:].reset_index(drop=True)
        df_p1 = df_p1.rename(columns={"Kabupaten / Kota": "Wilayah", "Unnamed: 1": "P1"})
        df_p1 = df_p1[['Wilayah', 'P1']]


    # 3. Variabel X5 (TPT)
    # File ini menggunakan "Kabupaten / Kota"
    file_tpt = f"{path_data_raw}Tingkat Pengangguran Terbuka (TPT) Menurut Kabupaten_Kota di Provinsi Jawa Tengah, {year}.csv"
    df_tpt = pd.read_csv(file_tpt) 
    df_tpt = df_tpt.iloc[2:].reset_index(drop=True)
    df_tpt = df_tpt[["Kabupaten / Kota", "Unnamed: 1"]]
    df_tpt = df_tpt.rename(columns={
        "Kabupaten / Kota": "Wilayah", 
        "Unnamed: 1": "TPT" 
    })


    # 4. Variabel X6 (Kepadatan Penduduk)
    # File ini menggunakan "Kabupaten dan Kota"
    file_kep = f"{path_data_raw}[Indikator Strategis] Kepadatan Penduduk Menurut Kabupaten_Kota di Provinsi Jawa Tengah, {year}.csv"
    df_kep = pd.read_csv(file_kep) 
    df_kep = df_kep.iloc[2:].reset_index(drop=True)
    df_kep = df_kep[["Kabupaten dan Kota", "Unnamed: 1"]]
    df_kep = df_kep.rename(columns={
        "Kabupaten dan Kota": "Wilayah", 
        "Unnamed: 1": "Kepadatan"
    })

    
    df_year = pd.merge(df_ipm, df_p1, on="Wilayah", how="inner")
    df_year = pd.merge(df_year, df_tpt, on="Wilayah", how="inner")
    df_year = pd.merge(df_year, df_kep, on="Wilayah", how="inner")
    
    df_year['Tahun'] = year
    
    all_dfs.append(df_year)
    print(f"Tahun {year} berhasil diproses dan digabung.")

df_combined = pd.concat(all_dfs, ignore_index=True)

df_combined = df_combined.dropna(subset=['Wilayah']) 
df_combined = df_combined[~df_combined['Wilayah'].str.contains("PROVINSI", na=False)] 

print("\nSukses menggabungkan semua 3 tahun.")
print(f"Total baris data: {len(df_combined)}")

df_combined.head(35)

Memulai proses untuk tahun: [2021, 2022, 2023]...

--> Memproses Tahun 2021...
Tahun 2021 berhasil diproses dan digabung.

--> Memproses Tahun 2022...
Tahun 2022 berhasil diproses dan digabung.

--> Memproses Tahun 2023...
Tahun 2023 berhasil diproses dan digabung.

Sukses menggabungkan semua 3 tahun.
Total baris data: 105


,Wilayah,UHH,HLS,RLS,Pengeluaran,P1,TPT,Kepadatan,Tahun
0,3301 Kabupaten Cilacap,73.9,12.63,7.09,10534,1.48,9.97,924,2021
1,3302 Kabupaten Banyumas,73.8,13.03,7.63,11546,2.35,6.05,1340,2021
2,3303 Kabupaten Purbalingga,73.21,12,7.25,10032,2.1,6.05,1487,2021
3,3304 Kabupaten Banjarnegara,74.28,11.63,6.75,9407,2.97,5.86,1003,2021
4,3305 Kabupaten Kebumen,73.55,13.35,7.55,9028,3.24,6.03,1124,2021
5,3306 Kabupaten Purworejo,74.87,13.51,8.21,10275,1.36,3.59,709,2021
6,3307 Kabupaten Wonosobo,71.94,11.76,6.82,10760,2.75,5.26,903,2021
7,3308 Kabupaten Magelang,73.88,12.55,7.79,9440,1.63,5.03,1184,2021
8,3309 Kabupaten Boyolali,76.03,12.57,7.85,13031,1.76,5.09,1061,2021
9,3310 Kabupaten Klaten,76.86,13.39,8.81,12017,1.79,5.48,1925,2021


### **4. Cleaning**

In [6]:
print("Memulai cleaning...")

# 1. Cleaning Nama Wilayah (menghapus kode 3301, 3302, dst.)
df_combined['Wilayah'] = df_combined['Wilayah'].astype(str).str.replace(r'^\d+\s+', '', regex=True).str.strip()
print("Nama wilayah dibersihkan (e.g., 'Kabupaten Cilacap')")

# 2. Tentukan Tipe Kolom
# 'Pengeluaran' dan 'Kepadatan' adalah angka utuh (integer)
int_cols = ['Pengeluaran', 'Kepadatan']
# Sisanya adalah angka desimal (float)
float_cols = ['UHH', 'HLS', 'RLS', 'TPT', 'P1']

print(f"Kolom Float: {float_cols}")
print(f"Kolom Integer: {int_cols}")

# 3. Cleaning Kolom Float (DESIMAL)
# GANTI koma (,) jadi titik (.), TAPI JANGAN HAPUS TITIK.
for col in float_cols:
    df_combined[col] = (
        df_combined[col]
        .astype(str)
        .str.replace(',', '.', regex=False) # Ganti koma jadi titik
        .str.strip()                        # Hapus spasi
        .astype(float)                      # Ubah ke float
    )
print("Kolom float berhasil di-cleaning.")

# 4. Cleaning Kolom Integer (ANGKA UTUH)
# HAPUS titik (.) atau spasi ( ) pemisah ribuan
for col in int_cols:
    df_combined[col] = (
        df_combined[col]
        .astype(str)
        .str.replace(r'[.\s]', '', regex=True) # Hapus titik ATAU spasi
        .astype(int)                           # Ubah ke integer
    )
print("Kolom integer berhasil di-cleaning.")

print("\n--> Cleaning Selesai")

Memulai cleaning...
Nama wilayah dibersihkan (e.g., 'Kabupaten Cilacap')
Kolom Float: ['UHH', 'HLS', 'RLS', 'TPT', 'P1']
Kolom Integer: ['Pengeluaran', 'Kepadatan']
Kolom float berhasil di-cleaning.
Kolom integer berhasil di-cleaning.

--> Cleaning Selesai


### **5. Validasi Data Bersih**

In [7]:
print("==================================")
print("=== Hasil Validasi Data Bersih ===")
print("==================================")

# 1. Cek tipe data
print("\nTipe Data Akhir:")
print(df_combined.dtypes)

# 2. Cek 5 data pertama
print("\nData Head (Cek Angka Desimal):")
print(df_combined.head())

# 3. Cek statistik deskriptif
print("\nStatistik Deskriptif (Cek Skala Angka):")
print(df_combined.describe())

# 4. Cek missing value
print("\nMissing Values:")
print(df_combined.isna().sum())

=== Hasil Validasi Data Bersih ===

Tipe Data Akhir:
Wilayah         object
UHH            float64
HLS            float64
RLS            float64
Pengeluaran      int64
P1             float64
TPT            float64
Kepadatan        int64
Tahun            int64
dtype: object

Data Head (Cek Angka Desimal):
                  Wilayah    UHH    HLS   RLS  Pengeluaran    P1   TPT  \
0       Kabupaten Cilacap  73.90  12.63  7.09        10534  1.48  9.97   
1      Kabupaten Banyumas  73.80  13.03  7.63        11546  2.35  6.05   
2   Kabupaten Purbalingga  73.21  12.00  7.25        10032  2.10  6.05   
3  Kabupaten Banjarnegara  74.28  11.63  6.75         9407  2.97  5.86   
4       Kabupaten Kebumen  73.55  13.35  7.55         9028  3.24  6.03   

   Kepadatan  Tahun  
0        924   2021  
1       1340   2021  
2       1487   2021  
3       1003   2021  
4       1124   2021  

Statistik Deskriptif (Cek Skala Angka):
              UHH         HLS         RLS   Pengeluaran          P1  \
count

### **6. Simpan Data Bersih**

In [8]:
# Susun ulang kolom
cols_ordered = [
    'Wilayah', 
    'Tahun', 
    'P1',  # Variabel Y
    'UHH', # Variabel X1
    'HLS', # Variabel X2
    'RLS', # Variabel X3
    'Pengeluaran', # Variabel X4
    'TPT', # Variabel X5
    'Kepadatan' # Variabel X6
]

df_final = df_combined[cols_ordered]

output_file = os.path.join(path_data_clean, "dataset_final_2021-2023.csv")

# Simpan ke CSV
df_final.to_csv(output_file, index=False)

print(f"\n--> SUKSES")
print(f"Data bersih final ({len(df_final)} baris, {len(df_final.columns)} kolom) telah disimpan di:")
print(output_file)


--> SUKSES
Data bersih final (105 baris, 9 kolom) telah disimpan di:
../data_clean/dataset_final_2021-2023.csv
